In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import os
import gc

from tensorflow.keras import layers, optimizers
from keras.models import Model
from keras.layers import Input,Dense,LSTM,Reshape,Dropout
from keras.utils import Sequence
import os, psutil 

In [ ]:
def cpu_stats():
    pid = os.getpid()
    py = psutil.Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return 'memory GB:' + str(np.round(memory_use, 2))

# Load and prepear dataset

In [ ]:
with open('/kaggle/input/asl-signs/sign_to_prediction_index_map.json') as f:
    sign_ids = json.load(f)

In [ ]:
# LANDMARK_IDX = [0,9,11,13,14,17,117,118,119,199,346,347,348] + list(range(468,543))
LIPS_IDXS =[0,9,11,13,14,17,37,39,40,61,78,80,81,82,84,87,88,91,95,117,118,119,
                146,178,181,185,191,199,267,269,270,291,308,310,311,312,314,317,318,321,
                324,346,347,348,375,402,405,409,415]
LEFT_HAND_IDXS = np.arange(468,489)

REF_POSE_IDXS=[489]
LEFT_POSE_IDXS=[490,491,492,496,498,500,502,504,506,508,510,512,514,516,518,520]
RIGHT_POSE_IDXS=[493,494,495,497,499,501,503,505,507,509,511,513,515,517,519,521]
DIFF_LEFT_POSE_IDXS=[1,2,3,7,9,11,13,15,17,19,21,23,25,27,29,31]
DIFF_RIGHT_POSE_IDXS=[4,5,6,8,10,12,14,16,18,20,22,24,26,28,30,32]
NEW_LEFT_POSE_IDXS=[490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505]
NEW_RIGHT_POSE_IDXS=[506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521]

POSE_IDXS = np.arange(489,522) #Se corrigio post flip para no intercalar los valores nuevamente
RIGHT_HAND_IDXS = np.arange(522,543)

LANDMARK_IDX = list(LIPS_IDXS)+list(POSE_IDXS)+list(LEFT_HAND_IDXS)+list(RIGHT_HAND_IDXS)

# LANDMARK_IDX = range(0,543)
DATA_COLUMNS    = ['x', 'y', 'z']
# DATA_COLUMNS    = ['x', 'y']
ROWS_PER_FRAME  = 543
NUM_SHARDS      = 2
SAVE_PATH       = '/tmp/GoogleISLDataset'
BATCH_SIZE  = 64
data_dir = "/kaggle/input/asl-signs"
N_FRAMES_NORM=64
CLASS_COUNT=250
LEARNING_RATE=0.001

In [ ]:
def load_relevant_data_subset(pq_path):
    data = pd.read_parquet('/kaggle/input/asl-signs/'+pq_path, columns=['x', 'y', 'z'])
    n_frames = int(len(data) / 543)
    data = data.values.astype(np.float32)
    return data.reshape(n_frames, 543, 3)

In [ ]:
def read_json(path):
    with open(path, "r") as file:
        json_data = json.load(file)
    return json_data

In [ ]:
df = pd.read_csv('/kaggle/input/asl-signs/train.csv')

In [ ]:
df.head(2)

In [ ]:
s2p_map = read_json(os.path.join(data_dir, "sign_to_prediction_index_map.json"))
p2s_map = {v: k for k, v in s2p_map.items()}

encoder = lambda x: s2p_map.get(x)
decoder = lambda x: p2s_map.get(x)

df["label"] = df["sign"].map(encoder)
print(f"shape = {df.shape}")

df=df.sample(frac=1,random_state=42)
df.reset_index(drop=True,inplace=True)
df.head(2)

In [ ]:
train_df,val_df = train_test_split(
    df, test_size=0.2, stratify=df['label'], random_state=42
)
train_df.reset_index(drop=True,inplace=True)
val_df.reset_index(drop=True,inplace=True)

In [ ]:
def get_rotation_matrix(degree):
    tita_rad=degree*np.pi/180
    R=np.array([[np.cos(tita_rad),-np.sin(tita_rad)],[np.sin(tita_rad),np.cos(tita_rad)]])
    return tf.cast(tf.constant(R), tf.float32)

def get_augmentation_x(x_samp,angle=0):
    # Original mass center | shape(frames, 124, 3) 1,frames,landmarks_id,xyz
    # Landmarks is the dimension to mean    
    x_samp_xy=tf.gather(x_samp, [0,1], axis=2) # --> shape(frames, 124, 2)
    mass_center_orig=tf.experimental.numpy.nanmean(x_samp_xy,axis=1) # --> shape(frames, 2)
    
    # Expand dim from sum with rotated tensor
    mass_center_orig = tf.expand_dims(mass_center_orig, axis=1) # --> shape(frames,1, 2)

    R=get_rotation_matrix(angle)  
    # Rotation
    # x_samp[:,:,:,[0, 1]]=np.matmul(x_samp[:,:,:,[0, 1]],R) 
    x_samp_rot_xy=tf.matmul(tf.cast(tf.gather(x_samp,tf.range(0,2), axis=2), tf.float32),R) # --> shape(frames, 124, 2)
    x_samp_rot_z=tf.gather(x_samp,tf.range(2,3), axis=2) # Really z is not rotated but the name is for normalization 
    
    # Rotation mass center
    mass_center_rot=tf.experimental.numpy.nanmean(x_samp_rot_xy,axis=1) # --> shape(frames, 2)

    # Expand dim from sum with rotated tensor
    mass_center_rot = tf.expand_dims(mass_center_rot, axis=1) # --> shape(frames,1, 2)

    # Mass center difference
    mass_center_dif=mass_center_orig-mass_center_rot    
    x_samp_rot_xy=x_samp_rot_xy+mass_center_dif
    
    # We have to concat to get a shape (None,len(LANDMARK_IDX),3)
    x_samp_rot = tf.concat([x_samp_rot_xy, x_samp_rot_z], axis=2)
    return x_samp_rot    

In [ ]:
def get_augm_flip(inputs):    
    out_center_flipx=tf.gather(inputs,[0],axis=-1)*(-1)
    out_center_flipyz=tf.gather(inputs,[1,2],axis=-1)
    out_center_flip = tf.concat([out_center_flipx,out_center_flipyz],axis=-1)
    
    tf_lips=tf.gather(out_center_flip,np.arange(0,468),axis=1) # Se tiene en cuenta toda la cabeza
    tf_lhand=tf.gather(out_center_flip,LEFT_HAND_IDXS,axis=1)
    tf_ref_pose=tf.gather(out_center_flip,REF_POSE_IDXS,axis=1)
    tf_lpose=tf.gather(out_center_flip,NEW_LEFT_POSE_IDXS,axis=1)
    tf_rpose=tf.gather(out_center_flip,NEW_RIGHT_POSE_IDXS,axis=1)
    tf_rhand=tf.gather(out_center_flip,RIGHT_HAND_IDXS,axis=1)
    
    out=tf.concat([tf_lips,tf_rhand,tf_ref_pose,tf_rpose,tf_lpose,tf_lhand],axis=1)
    
    return out

In [ ]:
def get_augm_time(x):
    # shape(frames, 124, 3)
    if (x.shape[0]<=6):
        action='slow'
        ratio=4        
    elif (x.shape[0]>6) & (x.shape[0]<=12):
        action='slow'
        ratio=2
    elif (x.shape[0]>32) & (x.shape[0]<=64):
        action='fast' 
        ratio=2
    elif (x.shape[0]>32) & (x.shape[0]<64):
        action='fast'  
        ratio=3
    elif (x.shape[0]>64) & (x.shape[0]<128):
        action='fast'  
        ratio=4
    elif (x.shape[0]>=128):
        action='fast'  
        ratio=8
    else:
        return x
        
    if action=='fast':
        accelerate=tf.range(0, x.shape[0], delta=ratio)
        x=tf.gather(x,accelerate,axis=0)
    elif action=='slow':
        x=tf.repeat(x,repeats=ratio,axis=0)
    else:
        return x 
    return x

In [ ]:
def normalize(tensor):
    tf_lips=tf.gather(tensor,np.arange(0,468),axis=1) # LIPS HARCODEADO
    tf_lhand=tf.gather(tensor,LEFT_HAND_IDXS,axis=1)
    tf_pose=tf.gather(tensor,np.arange(489,522),axis=1)
    tf_rhand=tf.gather(tensor,RIGHT_HAND_IDXS,axis=1)
    
    # LIPS
    tf_lips_x=tf.gather(tf_lips,[0],axis=-1)
    tf_lips_y=tf.gather(tf_lips,[1],axis=-1)
    tf_lips_z=tf.gather(tf_lips,[2],axis=-1)
    
    tf_lips=tf.concat([tf_lips_x-tf.gather(tf_lips_x,[0],axis=1),
                       tf_lips_y-tf.gather(tf_lips_y,[0],axis=1),
                       tf_lips_z-tf.gather(tf_lips_z,[0],axis=1)],axis=-1)
    
    # LHANDS
    tf_lhand_x=tf.gather(tf_lhand,[0],axis=-1)
    tf_lhand_y=tf.gather(tf_lhand,[1],axis=-1)
    tf_lhand_z=tf.gather(tf_lhand,[2],axis=-1)
    
    tf_lhand=tf.concat([tf_lhand_x-tf.gather(tf_lhand_x,[0],axis=1),
                       tf_lhand_y-tf.gather(tf_lhand_y,[0],axis=1),
                       tf_lhand_z-tf.gather(tf_lhand_z,[0],axis=1)],axis=-1)
    
    # POSE
    tf_pose_x=tf.gather(tf_pose,[0],axis=-1)
    tf_pose_y=tf.gather(tf_pose,[1],axis=-1)
    tf_pose_z=tf.gather(tf_pose,[2],axis=-1)
    
    tf_pose=tf.concat([tf_pose_x-tf.gather(tf_pose_x,[0],axis=1),
                       tf_pose_y-tf.gather(tf_pose_y,[0],axis=1),
                       tf_pose_z-tf.gather(tf_pose_z,[0],axis=1)],axis=-1)
 
    # RIGHT HANDS
    tf_rhand_x=tf.gather(tf_rhand,[0],axis=-1)
    tf_rhand_y=tf.gather(tf_rhand,[1],axis=-1)
    tf_rhand_z=tf.gather(tf_rhand,[2],axis=-1)
    
    tf_rhand=tf.concat([tf_rhand_x-tf.gather(tf_rhand_x,[0],axis=1),
                       tf_rhand_y-tf.gather(tf_rhand_y,[0],axis=1),
                       tf_rhand_z-tf.gather(tf_rhand_z,[0],axis=1)],axis=-1)
    
    tf_pose_ref=tf.gather(tf_pose,[0],axis=1)
    tf_pose_left=tf.gather(tf_pose,DIFF_LEFT_POSE_IDXS,axis=1)
    tf_pose_right=tf.gather(tf_pose,DIFF_RIGHT_POSE_IDXS,axis=1)
    
    out=tf.concat([tf_lips,tf_lhand,tf_pose_ref,tf_pose_left,tf_pose_right,tf_rhand],axis=1)
    return out

In [ ]:
def rotated_part(tensor,angle=0,part='lhand',plane='xy'):
    # SIEMPRE se debe poner despues de la funcion normalizada
    # Original mass center | shape(frames, 124, 3) 1,frames,landmarks_id,xyz
    tf_lips=tf.gather(tensor,np.arange(0,468),axis=1) # LIPS HARCODEADO
    tf_lhand=tf.gather(tensor,LEFT_HAND_IDXS,axis=1)
    tf_pose=tf.gather(tensor,np.arange(489,522),axis=1)
    tf_rhand=tf.gather(tensor,RIGHT_HAND_IDXS,axis=1)

    # Landmarks is the dimension to mean   
    plane_rot=tf.constant([0,1]) #x,y
    plane_fix=tf.constant([2])   #z
    
    if part=='lips':
        x_samp=tf_lips
    elif part=='lhand':
        x_samp=tf_lhand
    elif part=='pose':
        x_samp=tf_pose
    elif part=='rhand':
        x_samp=tf_rhand
    else:
        None
    
    x_samp_x=tf.gather(x_samp,[0], axis=2) # --> shape(frames, 124, 2)
    x_samp_y=tf.gather(x_samp,[1], axis=2) # --> shape(frames, 124, 2)
    x_samp_z=tf.zeros(shape=(x_samp.shape[0],x_samp.shape[1],1)) # --> shape(frames, 124, 2)
    
    if plane=='xy':
        x_to_rot=tf.concat([x_samp_x,x_samp_y],axis=2)
    elif plane=='yz':
        x_to_rot=tf.concat([x_samp_y,x_samp_z],axis=2)
    else: #'xz'
        x_to_rot=tf.concat([x_samp_x,x_samp_z],axis=2)

    R=get_rotation_matrix(angle)  
    # Rotation
    # x_samp[:,:,:,[0, 1]]=np.matmul(x_samp[:,:,:,[0, 1]],R) 
    x_samp_rot=tf.matmul(tf.cast(x_to_rot, tf.float32),R) # --> shape(frames, 124, 2)
    
    
    # We have to concat to get a shape (None,len(LANDMARK_IDX),3)
    if plane=='xy':
        x_samp_rot = tf.concat([x_samp_rot, x_samp_z], axis=2) #!OJO ACA CUANDO SE SUMA
    elif plane=='yz':
        x_samp_rot = tf.concat([x_samp_x, x_samp_rot], axis=2)
    else: #'xz'
        x_samp_rot_x=tf.gather(x_samp_rot,[0], axis=2)
        x_samp_rot_z=tf.gather(x_samp_rot,[1], axis=2)
        x_samp_rot = tf.concat([x_samp_rot_x,x_samp_y, x_samp_rot_z], axis=2)
        
    
    if part=='lips':
        out=tf.concat([x_samp_rot,tf_lhand,tf_pose,tf_rhand],axis=1)
    elif part=='lhand':
        out=tf.concat([tf_lips,x_samp_rot,tf_pose,tf_rhand],axis=1)
    elif part=='pose':
        out=tf.concat([tf_lips,tf_lhand,x_samp_rot,tf_rhand],axis=1)
    elif part=='rhand':
        out=tf.concat([tf_lips,tf_lhand,tf_pose,x_samp_rot],axis=1)
    else:
        out=None  
        
    return out 

# Preprocesing with Ragged Tensor

In [ ]:
def tf_get_features(ftensor):
    def feat_wrapper(ftensor):
        x=load_relevant_data_subset(ftensor.numpy().decode('utf-8'))
        return x
    return tf.py_function(
        feat_wrapper,
        [ftensor],
        Tout=tf.float32
    )

def tf_get_features_augm(ftensor,angle):
    def feat_wrapper(ftensor):
        x=get_augmentation_x(ftensor,angle)
        return x
    return tf.py_function(
        feat_wrapper,
        [ftensor],
        Tout=tf.float32
    )

def tf_get_features_augm_time(ftensor):
    def feat_wrapper(ftensor):
        x=get_augm_time(ftensor)
        return x
    return tf.py_function(
        feat_wrapper,
        [ftensor],
        Tout=tf.float32
    )

def tf_get_features_augm_flip(ftensor):
    def feat_wrapper(ftensor):
        x=get_augm_flip(ftensor)
        return x
    return tf.py_function(
        feat_wrapper,
        [ftensor],
        Tout=tf.float32
    )

def tf_get_features_normalize(ftensor):
    def feat_wrapper(ftensor):
        x=normalize(ftensor)
        return x
    return tf.py_function(
        feat_wrapper,
        [ftensor],
        Tout=tf.float32
    )

def tf_get_features_rotated_part(ftensor,angle=0,part='lhand',plane='xy'):
    def feat_wrapper(ftensor):
        x=rotated_part(ftensor,angle=0,part='lhand',plane='xy')
        return x
    return tf.py_function(
        feat_wrapper,
        [ftensor],
        Tout=tf.float32
    )

In [ ]:
class PreprocessLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(PreprocessLayer, self).__init__()
        
    @tf.function(
        input_signature=(tf.TensorSpec(shape=[None,len(LANDMARK_IDX),len(DATA_COLUMNS)], dtype=tf.float32),),
    )
    def call(self, data0):
        if len(DATA_COLUMNS)<3:
            data0 = tf.gather(data0, [0,1], axis=2)
        
        N_FRAMES0 = tf.shape(data0)[0]
        
        data=data0
        frames_landmark_non_nan_sum  = tf.math.reduce_sum(
                    tf.where(tf.math.is_nan(tf.gather(data0, tf.range(0,len(LANDMARK_IDX)), axis=1)), 0, 1),
                    axis=[1, 2],
                )
        non_empty_frames_idxs = tf.where(frames_landmark_non_nan_sum > 0)
        non_empty_frames_idxs = tf.squeeze(non_empty_frames_idxs, axis=1)
        data = tf.gather(data0, non_empty_frames_idxs, axis=0)
        
        # Cast Indices in float32 to be compatible with Tensorflow Lite
        non_empty_frames_idxs = tf.cast(non_empty_frames_idxs, tf.float32)
        # Normalize to start with 0
        non_empty_frames_idxs -= tf.reduce_min(non_empty_frames_idxs)
        
        # Number of Frames in Filtered Video
        N_FRAMES = tf.shape(data)[0]
#         data = tf.gather(data, LANDMARK_IDX, axis=1)
        
        if N_FRAMES < N_FRAMES_NORM:
            #-----------------------
            # Pad with reppited values
#             ratio=tf.math.floordiv(N_FRAMES_NORM,N_FRAMES)+1
#             data=tf.repeat(data,repeats=ratio,axis=0) # Could be bigger than N_FRAMES_NORM
#             tensor_index_frame=tf.range(0,N_FRAMES_NORM, delta=1, dtype=tf.int32, name='range')*N_FRAMES*ratio
#             tensor_index_frame=tf.math.floordiv(tensor_index_frame,N_FRAMES_NORM)
#             data = tf.gather(data, tensor_index_frame, axis=0)
            #------------------------
            # Pad Data With Zeros
            data = tf.pad(data, [[0, N_FRAMES_NORM-N_FRAMES], [0,0], [0,0]], constant_values=0)

        else:
            #--------------------
            # Elije los primeros frames
            tensor_index_frame=tf.range(0,N_FRAMES_NORM, delta=1, dtype=tf.int32) 
            # -----
            # Realiza un sampleo uniforme
#             tensor_index_frame=tf.range(0,N_FRAMES_NORM, delta=1, dtype=tf.int32, name='range')*N_FRAMES
#             tensor_index_frame=tf.math.floordiv(tensor_index_frame,N_FRAMES_NORM)
           
            data = tf.gather(data, tensor_index_frame, axis=0)
            
        #complete null with zeros
        data = tf.where(tf.math.is_nan(data), 0.0, data)
        
        data = tf.reshape(data,[N_FRAMES_NORM, len(LANDMARK_IDX)*len(DATA_COLUMNS)])

        return data
    
preprocess_layer = PreprocessLayer()

In [ ]:
class NormalizerLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(NormalizerLayer, self).__init__()
        
    @tf.function(
        input_signature=(tf.TensorSpec(shape=[None,543,3], dtype=tf.float32),),
    )
    def call(self, data):
        return normalize(data)
    
normalizer_layer = NormalizerLayer()

In [ ]:
# TRAIN
# ORIGINAL DATASET
tf_dataset_train_len=train_df['path'].shape[0]
print("Longitud original "+str(tf_dataset_train_len))

dataset_train=tf.data.Dataset.from_tensor_slices(list(train_df['path'].values))
dataset_train=dataset_train.map(lambda x:tf_get_features(x))
dataset_train=dataset_train.map(lambda x:tf.ensure_shape(x, tf.TensorShape([None, 543, 3])))
# CENTRA LAS PARTES DEL CUERPO
dataset_train=dataset_train.map(lambda x:tf_get_features_normalize(x)) 
#------------------------------------------------------------------------------------------------- # LABELS
labels_train = tf.data.Dataset.from_tensor_slices(train_df['label'].values.reshape(-1,1)) 
#------------------------------------------------------------------------------------------------- # LABELS
# COMLETA NULOS
dataset_train=dataset_train.map(lambda x:tf.where(tf.math.is_nan(x), tf.zeros_like(x), x))
# FLIP AUGMENTATION
frac_flip=1
random_state=16
dataset_augm_flip=tf.data.Dataset.from_tensor_slices(list(train_df['path'].sample(frac=frac_flip,random_state=random_state).values))
dataset_augm_flip=dataset_augm_flip.map(lambda x:tf_get_features(x))
dataset_augm_flip=dataset_augm_flip.map(lambda x:tf.ensure_shape(x, tf.TensorShape([None, 543, 3])))
dataset_augm_flip=dataset_augm_flip.map(lambda x:tf_get_features_augm_flip(x)) # Ya lo centra en el origen ademas de rotarlo
dataset_augm_flip=dataset_augm_flip.map(lambda x:tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)) 
dataset_train=dataset_train.concatenate(dataset_augm_flip)
#------------------------------------------------------------------------------------------------- # LABELS
labels_augm_flip = tf.data.Dataset.from_tensor_slices(train_df['label'].sample(frac=frac_flip,random_state=random_state).values.reshape(-1,1))
labels_train=labels_train.concatenate(labels_augm_flip)
#------------------------------------------------------------------------------------------------- # LABELS
tf_dataset_train_len=tf_dataset_train_len+train_df['path'].sample(frac=frac_flip,random_state=random_state).shape[0]

print("Post aumento flip "+str(tf_dataset_train_len))

# SPACIAL AUGMENTATION (ROTATION)
frac_dim=0.15
count_dim_augm=0

for k,angle in enumerate([-20,-10,10,20]):
    random_state=(k)*42
    dataset_augm_xy=tf.data.Dataset.from_tensor_slices(list(train_df['path'].sample(frac=frac_dim,random_state=random_state).values))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf_get_features(x))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf.ensure_shape(x, tf.TensorShape([None, 543, 3])))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)) # Es necesario para la rotacion
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf_get_features_rotated_part(x,angle=angle,part='lhand',plane='xy'))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf_get_features_rotated_part(x,angle=angle,part='rhand',plane='xy'))
    dataset_train=dataset_train.concatenate(dataset_augm_xy)
#------------------------------------------------------------------------------------------------- # LABELS    
    labels_augm_xy = tf.data.Dataset.from_tensor_slices(train_df['label'].sample(frac=frac_dim,random_state=random_state).values.reshape(-1,1))
    labels_train=labels_train.concatenate(labels_augm_xy)
#------------------------------------------------------------------------------------------------- # LABELS    
    count_dim_augm=count_dim_augm+1
for k,angle in enumerate([-20,20]):
    random_state=(k)*34
    dataset_augm_xy=tf.data.Dataset.from_tensor_slices(list(train_df['path'].sample(frac=frac_dim,random_state=random_state).values))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf_get_features(x))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf.ensure_shape(x, tf.TensorShape([None, 543, 3])))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)) # Es necesario para la rotacion
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf_get_features_rotated_part(x,angle=angle,part='lhand',plane='yz'))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf_get_features_rotated_part(x,angle=angle,part='rhand',plane='yz'))
    dataset_train=dataset_train.concatenate(dataset_augm_xy)
#------------------------------------------------------------------------------------------------- # LABELS    
    labels_augm_xy = tf.data.Dataset.from_tensor_slices(train_df['label'].sample(frac=frac_dim,random_state=random_state).values.reshape(-1,1))
    labels_train=labels_train.concatenate(labels_augm_xy)
#------------------------------------------------------------------------------------------------- # LABELS    
    count_dim_augm=count_dim_augm+1
for k,angle in enumerate([-20,20]):
    random_state=(k)*34
    dataset_augm_xy=tf.data.Dataset.from_tensor_slices(list(train_df['path'].sample(frac=frac_dim,random_state=random_state).values))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf_get_features(x))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf.ensure_shape(x, tf.TensorShape([None, 543, 3])))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)) # Es necesario para la rotacion
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf_get_features_rotated_part(x,angle=angle,part='lhand',plane='xz'))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf_get_features_rotated_part(x,angle=angle,part='rhand',plane='xz'))
    dataset_train=dataset_train.concatenate(dataset_augm_xy)
#------------------------------------------------------------------------------------------------- # LABELS    
    labels_augm_xy = tf.data.Dataset.from_tensor_slices(train_df['label'].sample(frac=frac_dim,random_state=random_state).values.reshape(-1,1))
    labels_train=labels_train.concatenate(labels_augm_xy)
#------------------------------------------------------------------------------------------------- # LABELS    
    count_dim_augm=count_dim_augm+1        
        
for k,angle in enumerate([-5,5]):
    random_state=k*42
    dataset_augm_xy=tf.data.Dataset.from_tensor_slices(list(train_df['path'].sample(frac=frac_dim,random_state=random_state).values))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf_get_features(x))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf.ensure_shape(x, tf.TensorShape([None, 543, 3])))
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)) # Es necesario para la rotacion
    dataset_augm_xy=dataset_augm_xy.map(lambda x:tf_get_features_rotated_part(x,angle=angle,part='pose',plane='xy'))
    dataset_train=dataset_train.concatenate(dataset_augm_xy) #concatena
#------------------------------------------------------------------------------------------------- # LABELS    
    labels_augm_xy = tf.data.Dataset.from_tensor_slices(train_df['label'].sample(frac=frac_dim,random_state=random_state).values.reshape(-1,1))
    labels_train=labels_train.concatenate(labels_augm_xy)
#------------------------------------------------------------------------------------------------- # LABELS    
    count_dim_augm=count_dim_augm+1   
    
for j,plane in enumerate(['xy','yz','xz']):
    for k,angle in enumerate([-10,10]):
        random_state=(j+k)*27
        dataset_augm_xy=tf.data.Dataset.from_tensor_slices(list(train_df['path'].sample(frac=frac_dim,random_state=random_state).values))
        dataset_augm_xy=dataset_augm_xy.map(lambda x:tf_get_features(x))
        dataset_augm_xy=dataset_augm_xy.map(lambda x:tf.ensure_shape(x, tf.TensorShape([None, 543, 3])))
        dataset_augm_xy=dataset_augm_xy.map(lambda x:tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)) # Es necesario para la rotacion
        dataset_augm_xy=dataset_augm_xy.map(lambda x:tf_get_features_rotated_part(x,angle=angle,part='lips',plane=plane))
        dataset_train=dataset_train.concatenate(dataset_augm_xy) #concatena
    #------------------------------------------------------------------------------------------------- # LABELS    
        labels_augm_xy = tf.data.Dataset.from_tensor_slices(train_df['label'].sample(frac=frac_dim,random_state=random_state).values.reshape(-1,1))
        labels_train=labels_train.concatenate(labels_augm_xy)
    #------------------------------------------------------------------------------------------------- # LABELS    
        count_dim_augm=count_dim_augm+1  
    
    
    
tf_dataset_train_len=tf_dataset_train_len+train_df['path'].sample(frac=frac_dim,random_state=random_state).shape[0]*count_dim_augm
print("Post aumento rot "+str(tf_dataset_train_len))


# TIME AUGMENTATION
frac_time=0.35
random_state=57
dataset_augm_time=tf.data.Dataset.from_tensor_slices(list(train_df['path'].sample(frac=frac_time,random_state=random_state).values))
dataset_augm_time=dataset_augm_time.map(lambda x:tf_get_features(x))
dataset_augm_time=dataset_augm_time.map(lambda x:tf.ensure_shape(x, tf.TensorShape([None, 543, 3])))
dataset_augm_time=dataset_augm_time.map(lambda x:tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)) # Es necesario para la rotacion
dataset_augm_time=dataset_augm_time.map(lambda x:tf_get_features_augm_time(x)) #'fast','slow'
dataset_train=dataset_train.concatenate(dataset_augm_time)
#------------------------------------------------------------------------------------------------- # LABELS
labels_augm_time = tf.data.Dataset.from_tensor_slices(train_df['label'].sample(frac=frac_time,random_state=random_state).values.reshape(-1,1))
labels_train=labels_train.concatenate(labels_augm_time) 
#------------------------------------------------------------------------------------------------- # LABELS

tf_dataset_train_len=tf_dataset_train_len+train_df['path'].sample(frac=frac_time,random_state=random_state).shape[0]

print("Post aumento time "+str(tf_dataset_train_len))


# CREATE BATCH SIZE
# dataset_train=dataset_train.map(lambda x:preprocess_layer(x))
# dataset_train=dataset_train.map(lambda x:tf.ensure_shape(x, tf.TensorShape([N_FRAMES_NORM,len(LANDMARK_IDX)*len(DATA_COLUMNS)])))
# dataset_train=dataset_train.map(lambda x:tf.reshape(x,[-1, len(LANDMARK_IDX)*len(DATA_COLUMNS)]))

dataset_train=dataset_train.map(lambda x:tf.gather(x, LANDMARK_IDX, axis=1))

dataset_train=dataset_train.map(lambda x:tf.gather(x, [0,1], axis=-1)) # Se remueve Z
DATA_COLUMNS= ['x', 'y']


dataset_train=dataset_train.map(lambda x:tf.reshape(x,[-1, len(LANDMARK_IDX)*len(DATA_COLUMNS)]))
dataset_train=dataset_train.take(BATCH_SIZE * (tf_dataset_train_len // BATCH_SIZE)).apply(tf.data.experimental.dense_to_ragged_batch(batch_size=BATCH_SIZE))

labels_train=labels_train.take(BATCH_SIZE * (tf_dataset_train_len // BATCH_SIZE)).batch(BATCH_SIZE)

In [ ]:
# for batch in dataset_train:
# #     print(batch.shape)
#     for tensor in batch:
# #         print(tensor.numpy()[14,0:3,:])
# #         print('---------')        
#         tensor=preprocess_layer(tensor)
#         print(tensor.shape)
# #         print(tensor.numpy()[15,0:3,:])
#     break

In [ ]:
# VALIDATION
tf_dataset_val_len=val_df['path'].shape[0]

dataset_val=tf.data.Dataset.from_tensor_slices(list(val_df['path'].values))
dataset_val=dataset_val.map(lambda x:tf_get_features(x))
dataset_val=dataset_val.map(lambda x:tf_get_features_normalize(x))
dataset_val=dataset_val.map(lambda x:tf.ensure_shape(x, tf.TensorShape([None, 543, 3])))
dataset_val=dataset_val.map(lambda x:tf.gather(x, LANDMARK_IDX, axis=1))
dataset_val=dataset_val.map(lambda x:tf.where(tf.math.is_nan(x), tf.zeros_like(x), x))
# dataset_val=dataset_val.map(lambda x:preprocess_layer(x))
# dataset_val=dataset_val.map(lambda x:tf.ensure_shape(x, tf.TensorShape([N_FRAMES_NORM,len(LANDMARK_IDX)*len(DATA_COLUMNS)])))
# dataset_val=dataset_val.map(lambda x:tf.reshape(x,[-1, len(LANDMARK_IDX)*len(DATA_COLUMNS)]))


dataset_val=dataset_val.map(lambda x:tf.gather(x, [0,1], axis=-1)) # Se remueve Z
DATA_COLUMNS= ['x', 'y']

dataset_val=dataset_val.map(lambda x:tf.reshape(x,[-1, len(LANDMARK_IDX)*len(DATA_COLUMNS)]))
dataset_val=dataset_val.take(BATCH_SIZE * (tf_dataset_val_len // BATCH_SIZE)).apply(tf.data.experimental.dense_to_ragged_batch(batch_size=BATCH_SIZE))

labels_val = tf.data.Dataset.from_tensor_slices(val_df['label'].values.reshape(-1,1)).take(BATCH_SIZE * (tf_dataset_val_len // BATCH_SIZE)).batch(BATCH_SIZE)

# Saving dataset

In [ ]:
# Sharding could be improved, as the distribution of elements in different shards should optimally be equal.
# Currently, it will be a sample from a uniform distribution because this is simple to implement
def shard_func(*_):
    return tf.random.uniform(shape=[], maxval=NUM_SHARDS, dtype=tf.int64)

In [ ]:
# dataset_train=dataset_train.take(4*64)
# labels_train=labels_train.take(4*64)

# dataset_val=dataset_val.take(4*64)
# labels_val=labels_val.take(4*64)

In [ ]:

# SAVE_PATH_TRAIN='/kaggle.com/datasets/cristiandeblasis/islrRaggedFe/train'
SAVE_PATH_TRAIN='/kaggle/working/train_dataset'
train_ds = tf.data.Dataset.zip((dataset_train, labels_train))
train_ds.prefetch(tf.data.AUTOTUNE).save(SAVE_PATH_TRAIN, shard_func=shard_func)

In [ ]:
# SAVE_PATH_VAL='/kaggle.com/datasets/cristiandeblasis/islrRaggedFe/val'
SAVE_PATH_VAL='/kaggle/working/val_dataset'
val_ds = tf.data.Dataset.zip((dataset_val, labels_val))
val_ds.prefetch(tf.data.AUTOTUNE).save(SAVE_PATH_VAL, shard_func=shard_func)

## Load Dataset

In [ ]:
class PreprocessBatch(tf.keras.layers.Layer):
    def __init__(self):
        super(PreprocessBatch, self).__init__()
        
    @tf.function(
        input_signature=(tf.TensorSpec(shape=[BATCH_SIZE,None,len(LANDMARK_IDX),len(DATA_COLUMNS)], dtype=tf.float32),),
    )
    def call(self, data):       
        N_FRAMES = tf.shape(data)[1]
        print(data.shape)
        if N_FRAMES < N_FRAMES_NORM:
            #-----------------------
            # Pad with reppited values
#             ratio=tf.math.floordiv(N_FRAMES_NORM,N_FRAMES)+1
#             data=tf.repeat(data,repeats=ratio,axis=1) # Could be bigger than N_FRAMES_NORM
#             tensor_index_frame=tf.range(0,N_FRAMES_NORM, delta=1, dtype=tf.int32, name='range')*N_FRAMES*ratio
#             tensor_index_frame=tf.math.floordiv(tensor_index_frame,N_FRAMES_NORM)
#             data = tf.gather(data, tensor_index_frame, axis=1) 
            #------------------------
            # Pad Data With Zeros
            data = tf.pad(data, [[0,0],[0, N_FRAMES_NORM-N_FRAMES], [0,0], [0,0]], constant_values=0)            
        else:
            #--------------------
            # Elije los primeros frames
            tensor_index_frame=tf.range(0,N_FRAMES_NORM, delta=1, dtype=tf.int32) 
            # -----
            # Realiza un sampleo uniforme
#             tensor_index_frame=tf.range(0,N_FRAMES_NORM, delta=1, dtype=tf.int32, name='range')*N_FRAMES
#             tensor_index_frame=tf.math.floordiv(tensor_index_frame,N_FRAMES_NORM)
           
            data = tf.gather(data, tensor_index_frame, axis=1)            
        
        data = tf.where(tf.math.is_nan(data), 0.0, data)
#         data = tf.gather(data, [0,1], axis=-1) # Se remueve Z
        data = tf.reshape(data,[-1,N_FRAMES_NORM, len(LANDMARK_IDX)*len(DATA_COLUMNS)])
        
        return data
    
preprocess_batch = PreprocessBatch()

In [ ]:

@tf.function
def preparation(ragged_batch):
    dense_batch = ragged_batch.to_tensor()
    dense_batch=preprocess_batch(dense_batch)
    return dense_batch


# # Load the data and split to train, validation
# train_ds = tf.data.Dataset.load(SAVE_PATH_TRAIN).map(lambda x,y:(preparation(x),y)).prefetch(tf.data.AUTOTUNE)

# # Load the data and split to train, validation
# val_ds = tf.data.Dataset.load(SAVE_PATH_VAL).map(lambda x,y:(preparation(x),y)).prefetch(tf.data.AUTOTUNE)

# Ragged
# # Load the data and split to train, validation
train_ds = tf.data.Dataset.load(SAVE_PATH_TRAIN).prefetch(tf.data.AUTOTUNE)

# Load the data and split to train, validation
val_ds = tf.data.Dataset.load(SAVE_PATH_VAL).prefetch(tf.data.AUTOTUNE)

In [ ]:
# for b in train_ds:
#     print(b[0].shape)
#     break

# Model definition

In [ ]:
# # Initiailizers
# INIT_HE_UNIFORM = tf.keras.initializers.he_uniform
# INIT_GLOROT_UNIFORM = tf.keras.initializers.glorot_uniform
# INIT_ZEROS = tf.keras.initializers.constant(0.0)
# # Activations
# GELU = tf.keras.activations.gelu

# class LandmarkEmbedding(tf.keras.Model):
#     def __init__(self, units, name):
#         super(LandmarkEmbedding, self).__init__(name=f'{name}_embedding')
#         self.units = units
        
#     def build(self, input_shape):
#         # Embedding for missing landmark in frame, initizlied with zeros
#         self.empty_embedding = self.add_weight(
#             name=f'{self.name}_empty_embedding',
#             shape=[self.units],
#             initializer=INIT_ZEROS,
#         )
#         # Embedding
#         self.dense = tf.keras.Sequential([
#             tf.keras.layers.Dense(self.units, name=f'{self.name}_dense_1', use_bias=False, kernel_initializer=INIT_GLOROT_UNIFORM),
#             tf.keras.layers.Activation(GELU),
#             tf.keras.layers.Dense(self.units, name=f'{self.name}_dense_2', use_bias=False, kernel_initializer=INIT_HE_UNIFORM),
#         ], name=f'{self.name}_dense')

#     def call(self, x):
#         return tf.where(
#                 # Checks whether landmark is missing in frame
#                 tf.reduce_sum(x, axis=2, keepdims=True) == 0,
#                 # If so, the empty embedding is used
#                 self.dense(x),#self.empty_embedding,
#                 # Otherwise the landmark data is embedded
#                 self.dense(x),
#             )

In [ ]:
# # Dense layer units for landmarks
# LIPS_UNITS = 384
# HANDS_UNITS = 384
# POSE_UNITS = 384
# # final embedding and transformer embedding size
# UNITS = 256

# class PositionalEmbedding(layers.Layer):
#     def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
#         super().__init__(**kwargs)
# #         self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)
#         self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=output_dim)
#         self.sequence_length = sequence_length
#         self.input_dim = input_dim
#         self.output_dim = output_dim
        
#         # Se incluye embedding for landmarks
#         # Embedding layer for Landmarks
#         self.lips_embedding = LandmarkEmbedding(LIPS_UNITS, 'lips')
#         self.left_hand_embedding = LandmarkEmbedding(HANDS_UNITS, 'left_hand')
#         self.pose_embedding = LandmarkEmbedding(POSE_UNITS, 'pose')
#         self.right_hand_embedding = LandmarkEmbedding(HANDS_UNITS, 'right_hand')
#         # Landmark Weights
#         self.landmark_weights = tf.Variable(tf.zeros([4], dtype=tf.float32), name='landmark_weights')
#         # Fully Connected Layers for combined landmarks
#         self.fc = tf.keras.Sequential([
#             tf.keras.layers.Dense(UNITS, name='fully_connected_1', use_bias=False, kernel_initializer=INIT_GLOROT_UNIFORM),
#             tf.keras.layers.Activation(GELU),
#             tf.keras.layers.Dense(UNITS, name='fully_connected_2', use_bias=False, kernel_initializer=INIT_HE_UNIFORM),
#         ], name='fc')
    
#     def call(self, lips0,left_hand0,pose0,right_hand0):
#         length = self.sequence_length# tf.shape(inputs)[-1]
#         positions = tf.range(start=0, limit=length, delta=1)
# #         embedded_tokens = self.token_embeddings(inputs)
        

#         # Lips
#         lips_embedding = self.lips_embedding(lips0)
#         # Left Hand
#         left_hand_embedding = self.left_hand_embedding(left_hand0)
#         # Pose
#         pose_embedding = self.pose_embedding(pose0)
#         # Right Hand
#         right_hand_embedding = self.right_hand_embedding(right_hand0)
#         # Merge Embeddings of all landmarks with mean pooling
#         x = tf.stack((
#             lips_embedding, left_hand_embedding, pose_embedding,right_hand_embedding
#         ), axis=3) #(None, 64, 384, 4)
#         print(x.shape)
#         x = x * tf.nn.softmax(self.landmark_weights) #(None, 64, 384, 4)
#         print(x.shape)
#         x = tf.reduce_sum(x, axis=3) #(None, 64, 384) 
#         print(x.shape)
#         # Fully Connected Layers
#         x = self.fc(x) #(None, 64, 256)
#         print(x.shape)
        

#         embedded_positions = self.position_embeddings(positions)
#         return x + embedded_positions
    
# #     def compute_mask(self, inputs, mask=None):
# # #         mask=tf.ones([1,16],dtype=tf.dtypes.float32)
# #         mask=tf.math.not_equal(tf.reduce_sum(inputs, axis=[-1], keepdims=False), 0)
# #         return mask
# #         return tf.math.not_equal(tf.reduce_sum(data, axis=[-1], keepdims=False), 0)
    
#     def get_config(self):
#         config = super().get_config()
#         config.update({"output_dim": self.output_dim,
#                        "sequence_length": self.sequence_length,
#                        "input_dim": self.input_dim})
#         return config

In [ ]:
# MLP_RATIO=2
# class TransformerEncoder(layers.Layer):
#     def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
#         super().__init__(**kwargs)
#         self.embed_dim = embed_dim
#         self.dense_dim = dense_dim
#         self.num_heads = num_heads
#         self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
# #         self.dense_proj = tf.keras.Sequential([layers.Dense(dense_dim, activation="relu",kernel_initializer=tf.keras.initializers.he_uniform),
# #                                                layers.Dense(embed_dim, kernel_initializer=tf.keras.initializers.glorot_uniform)])
#         self.dense_proj = tf.keras.Sequential([layers.Dense(UNITS*MLP_RATIO,activation=GELU,
#                                                             kernel_initializer=tf.keras.initializers.he_uniform),
#                                                layers.Dense(UNITS, 
#                                                             kernel_initializer=tf.keras.initializers.glorot_uniform)])        
#         self.layernorm_1 = layers.LayerNormalization()
#         self.layernorm_2 = layers.LayerNormalization()
# #         self.supports_masking = True #https://keras.io/guides/understanding_masking_and_padding/
        
#     def call(self, inputs, mask=None):
# #         if mask is not None:
# # #             mask = mask[:,tf.newaxis,:]
# #             mask=tf.expand_dims(mask, axis=2)       
        
#         attention_output = self.attention(inputs, inputs, attention_mask=mask)
#         proj_input = self.layernorm_1(inputs + attention_output)
#         proj_output = self.dense_proj(proj_input)
#         return self.layernorm_2(proj_input + proj_output)

#     def get_config(self):
#         config = super().get_config()
#         config.update({"embed_dim": self.embed_dim,"num_heads": self.num_heads,"dense_dim": self.dense_dim,})
#         return config

In [ ]:
# # sequence_length = 600
# embed_dim = UNITS
# num_heads = 4
# dense_dim = 64 
# sequence_length=N_FRAMES_NORM
# vocab_size=None

# transformer_layer1=TransformerEncoder(embed_dim, dense_dim, num_heads)
# transformer_layer2=TransformerEncoder(embed_dim, dense_dim, num_heads)

# inputs = tf.keras.Input(shape=(N_FRAMES_NORM,len(LANDMARK_IDX)*len(DATA_COLUMNS)), dtype="float32",name="input")

# mask1=tf.math.not_equal(tf.reduce_sum(inputs, axis=[-1], keepdims=False), 0)
# mask1=tf.expand_dims(mask1, axis=2)  

# lips0=tf.gather(inputs, tf.range(START_LIPS*len(DATA_COLUMNS),STOP_LIPS*len(DATA_COLUMNS)), axis=-1)
# left_hand0=tf.gather(inputs, tf.range(START_LHANDS*len(DATA_COLUMNS),STOP_LHANDS*len(DATA_COLUMNS)), axis=-1)
# pose0=tf.gather(inputs, tf.range(START_POSE*len(DATA_COLUMNS),STOP_POSE*len(DATA_COLUMNS)), axis=-1)
# right_hand0=tf.gather(inputs, tf.range(START_RHANDS*len(DATA_COLUMNS),STOP_RHANDS*len(DATA_COLUMNS)), axis=-1)

# x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(lips0,left_hand0,pose0,right_hand0)
# # x = layers.Embedding(vocab_size, embed_dim)(x)
# x = transformer_layer1(x,mask1) # Utiliza el vector de entrada como embeding
# # x = transformer_layer2(x,mask1) 
# # x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x) 
# # x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
# x = layers.GlobalMaxPooling1D()(x)
# x = layers.Dropout(0.2)(x)
# outputs = layers.Dense(CLASS_COUNT, activation="softmax")(x)
# model = tf.keras.Model(inputs, outputs)
# model.summary()

In [ ]:
input_tensor = Input(shape=(None,len(LANDMARK_IDX)*len(DATA_COLUMNS)), ragged=True)

x = layers.Dense(512)(input_tensor)
x = layers.LayerNormalization(axis=2)(x)
x = layers.Activation("relu")(x)
x = layers.Dropout(0.2)(x)

x = layers.Dense(256)(x)
x = layers.LayerNormalization(axis=2)(x)
x = layers.Activation("relu")(x)
x = layers.Dropout(0.2)(x)

x = layers.Dense(128)(x)
x = layers.LayerNormalization(axis=2)(x)
x = layers.Activation("relu")(x)
x = layers.Dropout(0.2)(x)

x = LSTM(96)(x)
output_tensor = Dense(CLASS_COUNT, activation='softmax')(x)

model = Model(input_tensor, output_tensor)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    metrics=["accuracy"],
)

model.summary(expand_nested=True)

In [ ]:
# Funciones de monitoreo

es_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=20, restore_best_weights=True
)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    "./LSTM_model",
    save_best_only=True,
    restore_best_weights=True,
    monitor="val_accuracy",
    mode="max",
    verbose=False,
)

cb_list = [checkpoint_callback,es_callback]

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              metrics=["accuracy"])

In [ ]:
model.fit(train_ds, validation_data=val_ds, epochs=80,callbacks = cb_list)

In [ ]:
# import scipy

# # Landmark Weights
# for w in model.get_layer('positional_embedding').weights:
#     if 'landmark_weights' in w.name:
#         weights = scipy.special.softmax(w)

# landmarks = ['lips_embedding', 'left_hand_embedding', 'pose_embedding','right_hand_embedding']

# for w, lm in zip(weights, landmarks):
#     print(w)
#     print(f'{lm} weight: {(w*100):.1f}%')

# Submit

In [ ]:
# model = tf.keras.models.load_model("./LSTM_model", custom_objects={'PositionalEmbedding': PositionalEmbedding})
model = tf.keras.models.load_model("./LSTM_model")

In [ ]:
# TFLite model for submission
class TFLiteModel(tf.Module):
    def __init__(self, model):
        super(TFLiteModel, self).__init__()

        # Load the feature generation and main models
#         self.preprocess_layer = preprocess_layer
        self.normalizer_layer = normalizer_layer
        self.model = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')])
    def __call__(self, inputs):
        # Preprocess Data
        x = self.normalizer_layer(inputs)
        x=tf.gather(x, [0,1], axis=-1)
        DATA_COLUMNS=['X','Y']
        x=tf.gather(x, LANDMARK_IDX, axis=1)
        x=tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
        x=tf.reshape(x,[-1, len(LANDMARK_IDX)*len(DATA_COLUMNS)])
        # Add Batch Dimension
        x = tf.expand_dims(x, axis=0)
     
        # Make Prediction
        outputs = self.model(x)
        print(outputs.shape)
        # Squeeze Output 1x250 -> 250
        outputs = tf.squeeze(outputs, axis=0)

        # Return a dictionary with the output tensor
        return {'outputs': outputs}

# Define TF Lite Model
tflite_keras_model = TFLiteModel(model)

# Sanity Check
# k=34
# demo_raw_data = load_relevant_data_subset(metadata.path[k])
# print(f'demo_raw_data shape: {demo_raw_data.shape}, dtype: {demo_raw_data.dtype}')
# demo_output = tflite_keras_model(demo_raw_data)["outputs"]
# print(f'demo_output shape: {demo_output.shape}, dtype: {demo_output.dtype}')
# demo_prediction = demo_output.numpy().argmax()
# print(f'demo_prediction: {demo_prediction}, correct: {s2p_map[metadata.sign[k]]}')

In [ ]:
# Create Model Converter
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflite_keras_model)
# Convert Model
tflite_model = keras_model_converter.convert()
# Write Model
with open('/kaggle/working/model.tflite', 'wb') as f:
    f.write(tflite_model)
    
# Zip Model
!zip submission.zip /kaggle/working/model.tflite

# Safe validation

In [ ]:
# !pip install tflite-runtime
# import tflite_runtime.interpreter as tflite
# import matplotlib.pyplot as plt

In [ ]:
# metadata = pd.read_csv("/kaggle/input/asl-signs/train.csv")
# with open("/kaggle/input/asl-signs/sign_to_prediction_index_map.json") as f:
#     sign_map = json.load(f)
# sign_list = list(sign_map.keys())

In [ ]:
# model_path = "model.tflite"
# interpreter = tflite.Interpreter(model_path)
# found_signatures = list(interpreter.get_signature_list().keys())
# # if REQUIRED_SIGNATURE not in found_signatures:
# #     raise KernelEvalException('Required input signature not found.')
# prediction_fn = interpreter.get_signature_runner("serving_default")

# y_trues = []
# y_preds = []
# for i in range(100):
#     data = load_relevant_data_subset(metadata.path[i])
#     output = prediction_fn(inputs=data)
#     sign_pred = np.argmax(output["outputs"])
    
#     y_trues.append(metadata.sign[i])
#     y_preds.append(sign_list[sign_pred])

In [ ]:
# y_num = [sign_map[sign] for sign in y_trues]
# y_prednum = [sign_map[sign] for sign in y_preds]
# confmat = tf.math.confusion_matrix(y_num,y_prednum)
# plt.imshow(confmat, cmap="binary")